In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade numpy mediapipe opencv-python-headless tensorflow pillow tqdm


  Using cached numpy-2.3.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached opencv_python_headless-4.12.0.88-cp37-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (19 kB)
  Using cached tensorflow-2.20.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.5 kB)
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.


In [ ]:
import os
import random
import shutil

input_dir = "/content/drive/MyDrive/DATASET_VSL/Frames_2_no_heic"
output_dir = "/content/drive/MyDrive/DATASET_VSL/sample_600"
num_samples = 600

#tạo thư mục ouput nếu chưa có
os.makedirs(output_dir, exist_ok = True)


# Duyệt từng chữ cái
for letter in sorted(os.listdir(input_dir)):
    letter_path = os.path.join(input_dir, letter)
    if not os.path.isdir(letter_path):
        continue
    # Lấy tất cả ảnh trong các thư mục con
    all_images = []
    for subfolder in os.listdir(letter_path):
        sub_path = os.path.join(letter_path, subfolder)
        if os.path.isdir(sub_path):
            for file in os.listdir(sub_path):
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    all_images.append(os.path.join(sub_path, file))

    print(f'{letter}: tìm thấy {len(all_images)} ảnh')

    # Chọn ngẫu nhiên 100 ảnh
    if len(all_images) <= num_samples:
        selected = all_images
    else:
        selected = random.sample(all_images, num_samples)

    # Tạo thư mục chữ cái trong output
    save_path = os.path.join(output_dir, letter)
    os.makedirs(save_path, exist_ok=True)
    # Copy ảnh
    for i, img_path in enumerate(selected, start=1):
        ext = os.path.splitext(img_path)[1]
        new_name = f"{letter}_{i:04d}{ext}"
        shutil.copy(img_path, os.path.join(save_path, new_name))

print("Hoàn tất! Dataset đã lưu tại:", output_dir)

A: tìm thấy 1367 ảnh
B: tìm thấy 902 ảnh
C: tìm thấy 809 ảnh
D: tìm thấy 558 ảnh
E: tìm thấy 1015 ảnh
G: tìm thấy 834 ảnh
H: tìm thấy 855 ảnh
I: tìm thấy 668 ảnh
K: tìm thấy 589 ảnh
L: tìm thấy 1011 ảnh
M: tìm thấy 627 ảnh
N: tìm thấy 719 ảnh
O: tìm thấy 646 ảnh
P: tìm thấy 905 ảnh
Q: tìm thấy 617 ảnh
R: tìm thấy 591 ảnh
S: tìm thấy 672 ảnh
T: tìm thấy 595 ảnh
U: tìm thấy 710 ảnh
V: tìm thấy 651 ảnh
X: tìm thấy 653 ảnh
Y: tìm thấy 730 ảnh
Đ: tìm thấy 945 ảnh
Hoàn tất! Dataset đã lưu tại: /content/drive/MyDrive/DATASET_VSL/sample_600


In [ ]:
# ==== CÀI ĐẶT: lần đầu cần !pip install mediapipe opencv-python ====
# !pip -q install mediapipe opencv-python

import os
import math
import numpy as np
from PIL import Image, ImageOps
from tqdm import tqdm
import mediapipe as mp

# ====== CẤU HÌNH ======
src_root = "/content/drive/MyDrive/DATASET_VSL/sample_600"        # input: 100 ảnh/nhãn
dst_root = "/content/drive/MyDrive/DATASET_VSL/600_resize"  # output
target_size = 500                     # ảnh đích 500x500
bbox_margin_ratio = 0.20              # thêm biên an toàn 20% quanh bàn tay
valid_exts = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

os.makedirs(dst_root, exist_ok=True)

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True,
                       max_num_hands=1,
                       min_detection_confidence=0.5)

def detect_hand_bbox(img_pil: Image.Image):
    """Trả về (x_min, y_min, x_max, y_max) theo pixel nếu thấy tay; ngược lại None."""
    img = ImageOps.exif_transpose(img_pil).convert("RGB")
    w, h = img.size
    img_np = np.array(img)  # RGB
    results = hands.process(img_np)

    if not results.multi_hand_landmarks:
        return None

    # gộp tất cả điểm mốc tay
    xs, ys = [], []
    for lm in results.multi_hand_landmarks[0].landmark:
        xs.append(lm.x * w)
        ys.append(lm.y * h)

    x_min, x_max = float(np.min(xs)), float(np.max(xs))
    y_min, y_max = float(np.min(ys)), float(np.max(ys))
    return x_min, y_min, x_max, y_max

def square_crop_around_bbox(w, h, bbox, margin_ratio=0.2):
    """Tạo crop vuông bao trọn bbox + margin, không vượt khỏi ảnh."""
    if bbox is None:
        # fallback: crop vuông ở giữa (không méo)
        side = min(w, h)
        left = (w - side) // 2
        top = (h - side) // 2
        return int(left), int(top), int(left + side), int(top + side)

    x0, y0, x1, y1 = bbox
    bw, bh = x1 - x0, y1 - y0
    # thêm biên an toàn
    cx, cy = (x0 + x1) / 2.0, (y0 + y1) / 2.0
    side = max(bw, bh)
    side = side * (1.0 + margin_ratio * 2.0)  # margin hai phía

    # đảm bảo cạnh không nhỏ hơn 1
    side = max(1.0, side)

    # khung vuông ban đầu
    left = cx - side / 2.0
    top = cy - side / 2.0
    right = cx + side / 2.0
    bottom = cy + side / 2.0

    # nếu vượt biên, tịnh tiến để khung nằm gọn trong ảnh
    if left < 0:
        right -= left
        left = 0
    if top < 0:
        bottom -= top
        top = 0
    if right > w:
        left -= (right - w)
        right = w
    if bottom > h:
        top -= (bottom - h)
        bottom = h

    # sau khi clamp, có thể bị không vuông do bị cấn biên -> làm vuông lần nữa
    side_final = min(right - left, bottom - top, w, h)
    # giữ trung tâm gần cx,cy nhưng vẫn nằm trong ảnh
    left = max(0, min(w - side_final, cx - side_final / 2.0))
    top = max(0, min(h - side_final, cy - side_final / 2.0))
    right = left + side_final
    bottom = top + side_final

    # ép kiểu int pixel
    return int(round(left)), int(round(top)), int(round(right)), int(round(bottom))

def process_image(path_in, path_out):
    with Image.open(path_in) as im:
        im = ImageOps.exif_transpose(im).convert("RGB")
        w, h = im.size
        bbox = detect_hand_bbox(im)  # None nếu không thấy tay
        crop_box = square_crop_around_bbox(w, h, bbox, margin_ratio=bbox_margin_ratio)
        cropped = im.crop(crop_box)  # không viền, chỉ cắt
        out = cropped.resize((target_size, target_size), Image.Resampling.LANCZOS)
        out.save(path_out, quality=95)

# ====== CHẠY QUA TỪNG NHÃN ======
for label in sorted(os.listdir(src_root)):
    src_label_dir = os.path.join(src_root, label)
    if not os.path.isdir(src_label_dir):
        continue
    dst_label_dir = os.path.join(dst_root, label)
    os.makedirs(dst_label_dir, exist_ok=True)

    files = [f for f in os.listdir(src_label_dir)
             if os.path.splitext(f)[1].lower() in valid_exts]

    print(f"Xử lý {label}: {len(files)} ảnh")
    for fname in tqdm(files):
        src_path = os.path.join(src_label_dir, fname)
        dst_path = os.path.join(dst_label_dir, fname)
        try:
            process_image(src_path, dst_path)
        except Exception as e:
            print(f"  [BỎ QUA] {src_path}: {e}")

print(" Hoàn tất! Ảnh 500x500  đã lưu tại:", dst_root)


Xử lý A: 600 ảnh


100%|██████████| 600/600 [00:48<00:00, 12.29it/s]


Xử lý B: 600 ảnh


100%|██████████| 600/600 [00:50<00:00, 11.90it/s]


Xử lý C: 600 ảnh


...

Xử lý Y: 600 ảnh


100%|██████████| 600/600 [00:48<00:00, 12.28it/s]


Xử lý Đ: 600 ảnh


100%|██████████| 600/600 [00:47<00:00, 12.65it/s]

 Hoàn tất! Ảnh 500x500  đã lưu tại: /content/drive/MyDrive/DATASET_VSL/600_resize


In [ ]:
# import os
# import shutil
# import numpy as np
# import cv2
# import mediapipe as mp
# import csv
# from PIL import Image, ImageOps
# from tqdm import tqdm
# from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img
# from collections import defaultdict

# # ========== CẤU HÌNH ==========
# input_dir = "/content/drive/MyDrive/DATASET_VSL/Frames_2_no_heic"
# processed_dir = "/content/processed_1080"            # Resize chuẩn 1080x1080
# augmented_dir = "/content/augmented_data"            # Sau khi tăng cường
# output_csv = "/content/hand_keypoints_dataset.csv"   # File chứa keypoints

# resize_size = (128, 128)
# aug_image_size = (128, 128)
# target_count = 400
# min_detection_conf = 0.7

In [ ]:
# === IMPORTS ===
import os
import numpy as np
from PIL import Image
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import (
    img_to_array, array_to_img, apply_affine_transform
)

# === CẤU HÌNH ===
processed_dir = "/content/drive/MyDrive/DATASET_VSL/600_resize"   # thư mục gốc
output_dir    = "/content/drive/MyDrive/DATASET_VSL/augmented_600_resize"
image_size    = (128, 128)

AUGMENTATIONS = [
    ("rotate_15",       lambda img: apply_affine_transform(img, theta=15)),
    ("rotate_-15",      lambda img: apply_affine_transform(img, theta=-15)),
    # ("flip",          lambda img: np.flip(img, axis=1)),
    ("brightness_down", lambda img: np.clip(img * 0.7, 0, 255).astype(np.uint8)),
    ("brightness_up",   lambda img: np.clip(img * 1.3, 0, 255).astype(np.uint8)),
]
NUM_AUGS_PER_IMAGE = len(AUGMENTATIONS)

valid_exts = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}
os.makedirs(output_dir, exist_ok=True)

def save_aug(img_array, name, dst_dir):
    img = array_to_img(img_array)
    img.save(os.path.join(dst_dir, name))

# === THỐNG KÊ ===
stats = {}

# === Duyệt từng lớp (A, B, C, …) ở mức 1 ===
classes = [d for d in sorted(os.listdir(processed_dir))
           if d.isalpha() and os.path.isdir(os.path.join(processed_dir, d))]

if not classes:
    print("⚠️ Không tìm thấy thư mục lớp (A,B,C,...) trong:", processed_dir)

for class_name in classes:
    class_dir = os.path.join(processed_dir, class_name)
    out_dir   = os.path.join(output_dir, class_name)
    os.makedirs(out_dir, exist_ok=True)

    # Gom tất cả ảnh trong lớp (kể cả nằm trong thư mục con)
    img_paths = []
    for r, _, files in os.walk(class_dir):
        for f in files:
            if os.path.splitext(f)[1].lower() in valid_exts:
                img_paths.append(os.path.join(r, f))

    if not img_paths:
        print(f" {class_name}: Không có ảnh hợp lệ.")
        stats[class_name] = {"in": 0, "out": 0}
        continue

    total_in = 0
    total_out = 0

    print(f"\n=== Lớp {class_name} — {len(img_paths)} ảnh gốc ===")
    for path in tqdm(img_paths, desc=f"Augment {class_name}", unit="img"):
        try:
            img = Image.open(path).convert("RGB").resize(image_size)
            base = img_to_array(img)

            # Lưu ảnh gốc
            fname = os.path.basename(path)
            save_aug(base, fname, out_dir)
            total_in  += 1
            total_out += 1

            # Lưu các bản tăng cường
            for tag, aug_fn in AUGMENTATIONS:
                aug = aug_fn(base)
                save_aug(aug, f"{tag}_{fname}", out_dir)
                total_out += 1

        except Exception as e:
            print(f"  [BỎ QUA] {path}: {e}")

    stats[class_name] = {"in": total_in, "out": total_out}
    print(f"[{class_name}] Ảnh gốc: {total_in} | Mỗi ảnh +{NUM_AUGS_PER_IMAGE} augment → Tổng: {total_out}")

# === TỔNG KẾT ===
grand_in  = sum(v["in"]  for v in stats.values())
grand_out = sum(v["out"] for v in stats.values())

print("\n===== BÁO CÁO TỔNG =====")
for k in sorted(stats.keys()):
    print(f"- {k}: in={stats[k]['in']}, out={stats[k]['out']}")
print(f"\nTỔNG CỘNG: Ảnh gốc={grand_in} | Sau augment={grand_out} (mỗi ảnh gốc + {NUM_AUGS_PER_IMAGE} augment)")

print("\n Hoàn tất! Ảnh đã lưu ở:", output_dir)



=== Lớp A — 599 ảnh gốc ===


Augment A: 100%|██████████| 599/599 [00:55<00:00, 10.84img/s]


[A] Ảnh gốc: 599 | Mỗi ảnh +4 augment → Tổng: 2995

=== Lớp B — 600 ảnh gốc ===


Augment B: 100%|██████████| 600/600 [01:01<00:00,  9.68img/s]


[B] Ảnh gốc: 600 | Mỗi ảnh +4 augment → Tổng: 3000

=== Lớp C — 600 ảnh gốc ===


...

[X] Ảnh gốc: 600 | Mỗi ảnh +4 augment → Tổng: 3000

=== Lớp Y — 600 ảnh gốc ===


Augment Y: 100%|██████████| 600/600 [00:56<00:00, 10.65img/s]


[Y] Ảnh gốc: 600 | Mỗi ảnh +4 augment → Tổng: 3000

=== Lớp Đ — 600 ảnh gốc ===


Augment Đ: 100%|██████████| 600/600 [00:55<00:00, 10.84img/s]

[Đ] Ảnh gốc: 600 | Mỗi ảnh +4 augment → Tổng: 3000

===== BÁO CÁO TỔNG =====
- A: in=599, out=2995
- B: in=600, out=3000
- C: in=600, out=3000
- D: in=558, out=2790
- E: in=600, out=3000
- G: in=600, out=3000
- H: in=600, out=3000
- I: in=600, out=3000
- K: in=589, out=2945
- L: in=600, out=3000
- M: in=600, out=3000
- N: in=600, out=3000
- O: in=600, out=3000
- P: in=600, out=3000
- Q: in=600, out=3000
- R: in=591, out=2955
- S: in=600, out=3000
- T: in=595, out=2975
- U: in=600, out=3000
- V: in=600, out=3000
- X: in=600, out=3000
- Y: in=600, out=3000
- Đ: in=600, out=3000

TỔNG CỘNG: Ảnh gốc=13732 | Sau augment=68660 (mỗi ảnh gốc + 4 augment)

 Hoàn tất! Ảnh đã lưu ở: /content/drive/MyDrive/DATASET_VSL/augmented_600_resize


In [ ]:
import os, csv
from collections import defaultdict
from tqdm import tqdm
import cv2
import mediapipe as mp

# === CẤU HÌNH ===
data_dir   = "/content/drive/MyDrive/DATASET_VSL/augmented_600_resize"  # hoặc set = output_dir bạn đã augment
output_csv = "/content/hand_keypoints_dataset_new.csv"
valid_exts = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}

# === THỐNG KÊ ===
valid_count = 0
invalid_count = 0
invalid_per_class = defaultdict(int)

# === HÀM LIỆT KÊ ẢNH (có sort để tái lập) ===
def list_images(folder):
    return sorted(
        [f for f in os.listdir(folder)
         if os.path.splitext(f)[1].lower() in valid_exts]
    )

# === XỬ LÝ ===
mp_hands = mp.solutions.hands
with mp_hands.Hands(static_image_mode=True,
                    max_num_hands=1,
                    min_detection_confidence=0.7) as hands, \
     open(output_csv, mode='w', newline='') as f:

    writer = csv.writer(f)
    # Header: label, x0,y0,z0,...,x20,y20,z20
    header = ['label'] + [f'{c}{i}' for i in range(21) for c in ['x','y','z']]
    writer.writerow(header)

    # Duyệt từng lớp
    for label in sorted(os.listdir(data_dir)):
        label_path = os.path.join(data_dir, label)
        if not os.path.isdir(label_path):
            continue

        img_files = list_images(label_path)
        if not img_files:
            continue

        print(f"Đang xử lý lớp {label} ({len(img_files)} ảnh)...")
        for img_name in tqdm(img_files, desc=f"Trích {label}", unit="img"):
            img_path = os.path.join(label_path, img_name)
            image = cv2.imread(img_path, cv2.IMREAD_COLOR)
            if image is None:
                invalid_count += 1
                invalid_per_class[label] += 1
                continue

            # BGR -> RGB
            rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(rgb)

            if results.multi_hand_landmarks:
                lm = results.multi_hand_landmarks[0]
                row = [label] + [v for p in lm.landmark for v in (p.x, p.y, p.z)]
                writer.writerow(row)
                valid_count += 1
            else:
                invalid_count += 1
                invalid_per_class[label] += 1

# === BÁO CÁO ===
print(f"\nTrích xuất xong! Ảnh hợp lệ: {valid_count:,} | Bị bỏ qua: {invalid_count:,}")
if invalid_per_class:
    print("Số ảnh bị bỏ qua theo lớp:")
    for k in sorted(invalid_per_class.keys()):
        print(f" - {k}: {invalid_per_class[k]}")
print("CSV đã lưu tại:", output_csv)


Đang xử lý lớp A (2995 ảnh)...


Trích A: 100%|██████████| 2995/2995 [03:00<00:00, 16.62img/s]


Đang xử lý lớp B (3000 ảnh)...


Trích B: 100%|██████████| 3000/3000 [03:01<00:00, 16.53img/s]


Đang xử lý lớp C (3000 ảnh)...


...

Đang xử lý lớp Y (3000 ảnh)...


Trích Y: 100%|██████████| 3000/3000 [03:07<00:00, 15.98img/s]


Đang xử lý lớp Đ (3000 ảnh)...


Trích Đ: 100%|██████████| 3000/3000 [03:10<00:00, 15.72img/s]


Trích xuất xong! Ảnh hợp lệ: 37,023 | Bị bỏ qua: 31,637
Số ảnh bị bỏ qua theo lớp:
 - A: 1615
 - B: 1199
 - C: 2056
 - D: 1221
 - E: 1191
 - G: 1193
 - H: 1144
 - I: 1185
 - K: 1021
 - L: 1188
 - M: 1855
 - N: 1965
 - O: 1434
 - P: 1469
 - Q: 1750
 - R: 1118
 - S: 1854
 - T: 1184
 - U: 1200
 - V: 1200
 - X: 1168
 - Y: 1200
 - Đ: 1227
CSV đã lưu tại: /content/hand_keypoints_dataset_new.csv
